In [1]:
import numpy as np
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model

<h1> CODE: UNDERSTANDING THE FLOW OF LAYERS FOR ONE CONVOLUTIONAL BLOCK</h1>

In [2]:
X_input = Input((226,226,3))

X = ZeroPadding2D((3,3))(X_input)

#### CONVOLUTIONAL BLOCK 1
Conv_1 = Conv2D(64, kernel_size=(7,7), strides=(2,2), kernel_initializer='he_normal', name='Conv_Block_1')(X)
Conv_1 = BatchNormalization(axis=3, name='Batch_Nm_block1')(Conv_1)
Conv_1 = Activation('relu', name='Activation_Block_1')(Conv_1)
Conv_1_output = MaxPooling2D((3,3), strides=(2,2), name='Output_Block_1')(Conv_1)


                                                #### CONVOLUTIONAL BOLCK 2

## CONVOLUTION BLOCK 2 ITERATION 1 (CONVOLUTIONAL LAYER/BLOCK)
shortcut = Conv_1_output 
Conv_2_1_a = Conv2D(64, (1,1), strides=(1,1), kernel_initializer='he_normal', name='Conv_Block2_1_a')(Conv_1_output)
Conv_2_1_a = BatchNormalization(axis=3, name='Batch_nm_block2_1_a')(Conv_2_1_a)
Conv_2_1_a = Activation('relu', name='Activation_Block_2_1_a')(Conv_2_1_a)
Conv_2_1_b = Conv2D(64, (3,3), strides=(1,1), kernel_initializer='he_normal', padding='same', name='Conv_block2_b')(Conv_2_1_a)
Conv_2_1_b = BatchNormalization(axis=3, name='Batch_nm_block2_b')(Conv_2_1_b)
Conv_2_1_b = Activation('relu', name='Activation_block_2_b')(Conv_2_1_b)
Conv_2_1_c = Conv2D(256, (1,1), strides=(1,1), kernel_initializer='he_normal')(Conv_2_1_b)
Conv_2_1_c = BatchNormalization(axis=3)(Conv_2_1_c)
Conv_2_1_shortcut = Conv2D(256, (1,1), strides=(1,1), kernel_initializer='he_normal')(shortcut)
Conv_2_1_shortcut = BatchNormalization(axis=3)(Conv_2_1_shortcut)
Conv_2_1_output = Add()([Conv_2_1_c, Conv_2_1_shortcut])
Conv_2_1_output = Activation('relu')(Conv_2_1_output)

## CONVOLUTION BLOCK 2 ITERATION 2 (IDENTITY LAYER/BLOCK)
Conv_2_2_shortcut = Conv_2_1_output
Conv_2_2_a = Conv2D(64, (1,1), strides=(1,1), kernel_initializer='he_normal')(Conv_2_1_output)
Conv_2_2_a = BatchNormalization(axis=3)(Conv_2_2_a)
Conv_2_2_a = Activation('relu')(Conv_2_2_a)
Conv_2_2_b = Conv2D(64, (3,3), strides=(1,1), kernel_initializer='he_normal', padding='same')(Conv_2_2_a)
Conv_2_2_b = BatchNormalization(axis=3)(Conv_2_2_b)
Conv_2_2_b = Activation('relu')(Conv_2_2_b)
Conv_2_2_c = Conv2D(256, (1,1), strides=(1,1), kernel_initializer='he_normal')(Conv_2_2_b)
Conv_2_2_c = BatchNormalization(axis=3)(Conv_2_2_c)
Conv_2_2_output = Add()([Conv_2_2_c, Conv_2_2_shortcut])
Conv_2_2_output = Activation('relu')(Conv_2_2_output)
                                
## CONVOLUTION BLOCK 2 ITERATION 3
### SAME AS CONVOLUTIONAL BLOCK 2 ITERATION 2


                                                    #### CONVOLUTIONAL BLOCK 3

## CONVOLUTIONAL BLOCK 3 ITERATION 1
shortcut = Conv_2_2_output
Conv_3_1_a = Conv2D(128, (1,1), strides=(2,2), kernel_initializer='he_normal')(Conv_2_2_output)
Conv_3_1_a = BatchNormalization(axis=3)(Conv_3_1_a)
Conv_3_1_a = Activation('relu')(Conv_3_1_a)
Conv_3_1_b = Conv2D(128, (3,3), strides=(1,1), kernel_initializer='he_normal', padding='same')(Conv_3_1_a)
Conv_3_1_b = BatchNormalization(axis=3)(Conv_3_1_b)
Conv_3_1_b = Activation('relu')(Conv_3_1_b)
Conv_3_1_c = Conv2D(512, (1,1), strides=(1,1), kernel_initializer='he_normal')(Conv_3_1_b)
Conv_3_1_c = BatchNormalization(axis=3)(Conv_3_1_c)
Conv_3_1_shortcut = Conv2D(512, (1,1), strides=(2,2), kernel_initializer='he_normal')(shortcut)
Conv_3_1_shortcut = BatchNormalization(axis=3)(Conv_3_1_shortcut)
Conv_3_1_output = Add()([Conv_3_1_c, Conv_3_1_shortcut])
Conv_3_1_output = Activation('relu')(Conv_3_1_output)

## CONVOLUTIONAL BLOCK 3 ITERATION 2
### SIMILAR TO IDENTITY BLOCK/LAYER OF CONVOLUTIONAL BLOCK 2

In [3]:
model = Model(inputs=X_input, outputs=Conv_3_1_output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 226, 226, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 232, 232, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv_Block_1 (Conv2D)           (None, 113, 113, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
Batch_Nm_block1 (BatchNormaliza (None, 113, 113, 64) 256         Conv_Block_1[0][0]               
______________________________________________________________________________________________

<h1> DEFINING FUNCTIONS AND CALLING THEM TO CREATE OUR RESIDUAL NETWORK</h1>

<h4>DEFINING A CONVOLUTIONAL LAYER</h4>

In [4]:
def conv_layer(inputs, filters, stride):
    
    shortcut = inputs
    
    f1, f2, f3 = filters
    
    conv_a = Conv2D(f1, (1,1), strides=(stride,stride))(inputs)
    conv_a = BatchNormalization(axis=3)(conv_a)
    conv_a = Activation('relu')(conv_a)
    
    conv_b = Conv2D(f2, (3,3), strides=(1, 1), padding='same')(conv_a)
    conv_b = BatchNormalization(axis=3)(conv_b)
    conv_b = Activation('relu')(conv_b)
    
    conv_c = Conv2D(f3, (1,1), strides=(1,1))(conv_b)
    conv_c = BatchNormalization(axis=3)(conv_c)
    
    shortcut = Conv2D(f3, (1,1), strides=(stride, stride))(shortcut)
    shortcut = BatchNormalization(axis=3)(shortcut)
    
    output = Add()([conv_c, shortcut])
    output = Activation('relu')(output)
    
    return output

<h4>DEFINING AN IDENTITY LAYER</h4>

In [5]:
def identity_layer(inputs, filters):
    
    shortcut = inputs
    
    f1, f2, f3 = filters
    
    conv_a = Conv2D(f1, (1,1), strides=(1,1))(inputs)
    conv_a = BatchNormalization(axis=3)(conv_a)
    conv_a = Activation('relu')(conv_a)
    
    conv_b = Conv2D(f2, (3,3), strides=(1,1), padding='same')(conv_a)
    conv_b = BatchNormalization(axis=3)(conv_b)
    conv_b = Activation('relu')(conv_b)
    
    conv_c = Conv2D(f3, (1,1), strides=(1,1))(conv_b)
    conv_c = BatchNormalization(axis=3)(conv_c)
    
    output = Add()([conv_c, shortcut])
    output = Activation('relu')(output)
    
    return output

<h4>DEFINING THE MODEL</h4>

In [6]:
def ResNet50(input_shape, classes):
    
    inputs = Input(input_shape)
    
    zero_padding = ZeroPadding2D((3,3))(inputs)
    
    #### CONVOLUTION BLOCK 1
    conv_1 = Conv2D(64, (7,7), strides=(2,2))(zero_padding)
    conv_1 = BatchNormalization(axis=3)(conv_1)
    conv_1 = Activation('relu')(conv_1)
    conv_1 = ZeroPadding2D((1,1))(conv_1)
    conv_1 = MaxPooling2D((3,3), strides=(2,2))(conv_1)
    
    #### CONVOLUTIONAL BLOCK 2
    conv_2 = conv_layer(conv_1, [64, 64, 256], stride=1)
    conv_2 = identity_layer(conv_2, [64,64,256])
    conv_2 = identity_layer(conv_2, [64, 64, 256])
    
    #### CONVOLUTIONAL BLOCK 3
    conv_3 = conv_layer(conv_2, [128, 128, 512], stride=2)
    conv_3 = identity_layer(conv_3, [128, 128, 512])
    conv_3 = identity_layer(conv_3, [128, 128, 512])
    conv_3 = identity_layer(conv_3, [128, 128, 512])
    
    #### CONVOLUTIONAL BLOCK 4
    conv_4 = conv_layer(conv_3, [256, 256, 1024], stride=2)
    conv_4 = identity_layer(conv_4, [256, 256, 1024])
    conv_4 = identity_layer(conv_4, [256, 256, 1024])
    conv_4 = identity_layer(conv_4, [256, 256, 1024])
    conv_4 = identity_layer(conv_4, [256, 256, 1024])
    conv_4 = identity_layer(conv_4, [256, 256, 1024])
    
    #### CONVOLUTIONAL BLOCK 5
    conv_5 = conv_layer(conv_4, [512, 512, 2048], stride=2)
    conv_5 = identity_layer(conv_5, [512, 512, 2048])
    conv_5 = identity_layer(conv_5, [512, 512, 2048])
   
    #### BASEMODEL TILL HERE
    
    #### HEAD MODEL FROM HERE - YOU CAN CHANGE THE HEAD MODEL AS YOU WISH
    
#     #### AVERAGE POOLING
    avg_pooling = GlobalAveragePooling2D()(conv_5)
    
    output = Dense(classes, activation='softmax')(avg_pooling)
    
    model_resnet50 = Model(inputs=inputs, outputs=avg_pooling)
    
    return model_resnet50

By increasing the number of identity layers in CONVOLUTIONAL BLOCK 3 and 4:

1. ResNet101 ==> 1 conv_layer and 22 identity_layer (CONVOLUTIONAL BLOCK 4)
2. ResNet152 ==> 1 conv_layer and 7 identity_layer (CONVOLUTIONAL BLOCK 3); 1 conv_layer and 35 identity_layer (CONVOLUTIONAL BLOCK 4)

In [8]:
model = ResNet50(input_shape=(224,224,3), classes=2)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 112, 112, 64) 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
batch_normalization_62 (BatchNo (None, 112, 112, 64) 256         conv2d_62[0][0]                  
____________________________________________________________________________________________

In [ ]:
from tensorflow.keras.applications import InceptionV3

In [ ]:
model = InceptionV3(include_top=False)

In [ ]:
weights